# Decrypting Ciphertext Using Markov Chain Monte Carlo

------------
### Background

This project is based on an application of Markov Chain Monte Carlo (MCMC) to cryptography, introduced by Persi Diaconis (2008). Here, MCMC was used to decrypt coded messages from a state prison.

The prison's coded messages (or _ciphertext_) were assumed to be a simple substitution cipher. In these ciphers, the original message $m$ (_plaintext_) is scrambled using a permutation $f$, one letter at a time, yielding the ciphertext $c$. 

For example, if the plaintext alphabet is `ABC` and the ciphertext alphabet is `XYZ`, then the message $m=$ `AABBCAA` is encrypted as $c=$ `XXYYZXX`.

--------
### Method

Markov Chain Monte Carlo (MCMC) methods are widely used to obtain samples from complicated, high-dimensional probability distributions. 

For the problem of decryption, we seek a bijection $f$ (also called a _key_) that maps characters in the _code space_ to those in our _regular alphabet_.

$$ f:\{\text{code space}\}\longrightarrow \{\text{regular alphabet}\}$$

A common approach to guess this mapping is for the cryptanalyst to look at the [frequency distribution](https://en.wikipedia.org/wiki/Frequency_analysis) of letters in the ciphertext and compare that to the frequency distribution of the natural language. 

This project's method models text as a string of characters from a first-order Markov process. Using the bigram character frequency from a reference text, a first-order transition matrix from character-to-character can be constructed. For example, a transition from 'q' to 'a' might rarely occur compared to going from 'q' to 'u'.

We define the _plausibility function_ as a function that takes a mapping $f$, character transition matrix $P$, and returns the likelihood of the decrypted text:

$$Pl(f) = \prod_{i=1}^{n-1}P(f(s_{i}),f(s_{i+1}))$$


For an alphabet consisting of $p$ characters, there are $p!$ possible mapping functions (states) - for any practical $p$ this is intractable to search. 

A time-reversible, ergodic Markov chain can be constructed to ensure a stationary distribution exists and is unique. This is done using a simple, symmetric proposal distribution: two randomly chosen letters are swapped. We include the possibility that a letter is swapped with itself to ensure the chain is aperiodic.

A Bayesian derivation of this set-up is that we are trying to sample from a posterior distribution and find its mode. We can score each sample and take the one with the highest plausibility score to find the best mapping. For some mapping $f_{i}$ we apply Bayes' rule, assuming a uniform prior on $f$:

$$P(f_{i}|ciphertext) = \frac{P(ciphertext|f_{i})P(f_{i})}{P(ciphertext)}=\frac{P(ciphertext|f_{i})(1/p!)}{\sum_{k=1}^{p!}{P(ciphertext|f_{k})P(f_{k})}}=\frac{P(ciphertext|f_{i})(1/p!)}{(1/p!)\sum_{k=1}^{p!}{P(ciphertext|f_{k})}}=\frac{P(ciphertext|f_{i})}{\sum_{k=1}^{p!}{P(ciphertext|f_{k})}} = \frac{Pl(f_{i})}{Z}$$

The Metropolis-Hastings algorithm is implemented which allows us to draw samples from the posterior without calculating the normalizing constant $Z$ in the denominator.

--------
# References
Diaconis, Persi. "The markov chain monte carlo revolution." Bulletin of the American Mathematical Society 46.2 (2009): 179-205.